In [ ]:
import pandas as pd

file_path = './data_retail'

In [ ]:
# Leer archivo csv 
df_customer = pd.read_csv(f'{file_path}/csv/customers.csv')
df_customer.head()

In [ ]:
# Leer archivo sin cabeceras y con separador
df_customer = pd.read_csv(f'{file_path}/customers', sep='|', header=None, names=["customer_id","customer_fname","customer_lname","customer_email","customer_password","customer_street","customer_city","customer_state","customer_zipcode"])
df_customer.head()

In [ ]:
# Leer archivo json
df_customer = pd.read_json(f'{file_path}/json/customers.json', lines=True)
df_customer.head()

In [ ]:
import pandas as pd

# ------------------------------
# 1. Carga de Datos
# ------------------------------

# Cargar datos desde archivos CSV o JSON (simulados como ejemplo)
customers = pd.read_csv(f"{file_path}/csv/customers.csv")
departments = pd.read_csv(f"{file_path}/csv/departments.csv")
categories = pd.read_csv(f"{file_path}/csv/categories.csv")
products = pd.read_csv(f"{file_path}/csv/products.csv")
orders = pd.read_csv(f"{file_path}/csv/orders.csv")
order_items = pd.read_csv(f"{file_path}/csv/order_items.csv")

In [ ]:
# ------------------------------
# 2. Exploración Inicial
# ------------------------------

customers.head()


In [ ]:

print(customers.info())

In [ ]:
print(orders["order_status"].value_counts())


In [ ]:
# ------------------------------
# 3. Limpieza y Preparación de Datos
# ------------------------------

# Eliminar duplicados si los hay
customers.drop_duplicates(inplace=True)

In [ ]:
# Verificar valores nulos
df_nulls = customers.isnull().sum()
print("\nValores nulos por columna:\n", df_nulls)

In [ ]:
# ------------------------------
# 4. Validaciones Avanzadas
# ------------------------------

# Validar correos electrónicos
import re
customers["email_valido"] = customers["customer_email"].apply(lambda x: bool(re.match(r"[^@]+@[^@]+\.[^@]+", str(x))))

# Validar que precios de productos sean positivos
productos_invalidos = products[products["product_price"] <= 0]
print("\nProductos con precios inválidos:\n", productos_invalidos)

# Validar que no haya IDs duplicados en claves primarias
assert customers["customer_id"].is_unique, "Error: IDs duplicados en customer_id"
assert products["product_id"].is_unique, "Error: IDs duplicados en product_id"

# Validar relaciones entre tablas (por ejemplo, productos referenciados en order_items deben existir)
productos_referenciados = order_items["order_item_product_id"].isin(products["product_id"])
assert productos_referenciados.all(), "Error: Hay productos en order_items que no existen en products"


In [ ]:
# ------------------------------
# 5. Enriquecimiento de Datos (Joins)
# ------------------------------

# Unir productos con sus categorías y departamentos
products_full = products.merge(categories, left_on="product_category_id", right_on="category_id")
products_full = products_full.merge(departments, left_on="category_department_id", right_on="department_id")


In [ ]:

# Unir ordenes con clientes
orders_customers = orders.merge(customers, left_on="order_customer_id", right_on="customer_id")

In [ ]:

# Unir order_items con productos
order_details = order_items.merge(products, left_on="order_item_product_id", right_on="product_id")

In [ ]:
# ------------------------------
# 6. Agregaciones y Análisis
# ------------------------------

# Total de ingresos por producto
ventas_por_producto = order_details.groupby("product_name")["order_item_subtotal"].sum().reset_index()
ventas_por_producto = ventas_por_producto.sort_values(by="order_item_subtotal", ascending=False)


In [ ]:
# Total de ventas por departamento
order_details_full = order_details.merge(products_full, on="product_id")
ventas_por_departamento = order_details_full.groupby("department_name")["order_item_subtotal"].sum().reset_index()
ventas_por_departamento = ventas_por_departamento.sort_values(by="order_item_subtotal", ascending=False)


In [ ]:
# ------------------------------
# 7. Creación de Funciones Personalizadas
# ------------------------------

def clasificar_estado_pedido(estado):
    if estado in ["CLOSED", "COMPLETE"]:
        return "Finalizado"
    elif estado in ["PENDING", "PROCESSING"]:
        return "En Proceso"
    else:
        return "Otro"

orders_customers["estado_clasificado"] = orders_customers["order_status"].apply(clasificar_estado_pedido)


In [ ]:
# ------------------------------
# 8. Exportación de Resultados
# ------------------------------

ventas_por_producto.to_csv(f"{file_path}/csv/ventas_por_producto.csv", index=False)
ventas_por_departamento.to_json(f"{file_path}/json/ventas_por_departamento.json", orient="records", lines=True)


In [ ]:
# ------------------------------
# 9. Buenas Prácticas Finales
# ------------------------------
# - Uso de nombres descriptivos para las variables
# - Uso de funciones para tareas repetitivas
# - Documentar el código con comentarios
# - Separar la carga, transformación y exportación